In [4]:
filesPath = "/home/anil/Anil_Documents/01.02.BID_DATA/PySPark/Spark-The-Definitive-Guide-master/data"

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
df_sales = spark.read.format("csv")\
.option("header","true")\
.option("schema","true")\
.load(filesPath + "/retail-data/all/online-retail-dataset.csv")



In [38]:
spark.conf.set("spark.sql.shuffle.partitions",30)

In [39]:
df_sales.where(expr("InvoiceNo = '554084' AND StockCode = '23298' ")).show(truncate=False)

+---------+---------+-----------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description      |Quantity|InvoiceDate    |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------+--------+---------------+---------+----------+--------------+
|554084   |23298    |BUNTING , SPOTTY |3       |5/22/2011 11:52|4.95     |12909     |United Kingdom|
|554084   |23298    |SPOTTY BUNTING   |3       |5/22/2011 11:52|4.95     |12909     |United Kingdom|
+---------+---------+-----------------+--------+---------------+---------+----------+--------------+



In [40]:
df_sales_agg = df_sales.groupBy("InvoiceNo","StockCode","Description","CustomerID","Country")\
.agg(sum("Quantity").alias("Sum_Quantity"), avg("UnitPrice").alias("Avg_UnitPrice"),max("InvoiceDate").alias("Max_InvDate"))

In [41]:
df_sales_agg.groupBy("InvoiceNo","StockCode").count()\
.where("count > 1")\
.show()

+---------+---------+-----+
|InvoiceNo|StockCode|count|
+---------+---------+-----+
|   575335|    23203|    2|
|   554084|    23298|    2|
+---------+---------+-----+



In [42]:
df_sales_agg.count()

531227

In [43]:
df_sales_agg.explain()

== Physical Plan ==
SortAggregate(key=[InvoiceNo#36, StockCode#37, Description#38, CustomerID#42, Country#43], functions=[sum(cast(Quantity#39 as double)), avg(cast(UnitPrice#41 as double)), max(InvoiceDate#40)])
+- *(2) Sort [InvoiceNo#36 ASC NULLS FIRST, StockCode#37 ASC NULLS FIRST, Description#38 ASC NULLS FIRST, CustomerID#42 ASC NULLS FIRST, Country#43 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(InvoiceNo#36, StockCode#37, Description#38, CustomerID#42, Country#43, 30)
      +- SortAggregate(key=[InvoiceNo#36, StockCode#37, Description#38, CustomerID#42, Country#43], functions=[partial_sum(cast(Quantity#39 as double)), partial_avg(cast(UnitPrice#41 as double)), partial_max(InvoiceDate#40)])
         +- *(1) Sort [InvoiceNo#36 ASC NULLS FIRST, StockCode#37 ASC NULLS FIRST, Description#38 ASC NULLS FIRST, CustomerID#42 ASC NULLS FIRST, Country#43 ASC NULLS FIRST], false, 0
            +- *(1) FileScan csv [InvoiceNo#36,StockCode#37,Description#38,Quantity#39,InvoiceD

In [44]:
df_sales_agg.rdd.getNumPartitions()

30

In [45]:
df_sales.rdd.getNumPartitions()

2